In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 17 01:20:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    31W /  70W |      0MiB / 15360MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%%capture
!apt install git-lfs

In [ ]:
from datasets import load_dataset, load_metric, Audio

# probably takes 25-30 min
ami_orig = load_dataset("edinburghcstr/ami", "ihm")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ami downloaded and prepared to /root/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ami_orig

DatasetDict({
    train: Dataset({
        features: ['meeting_id', 'audio_id', 'text', 'audio', 'begin_time', 'end_time', 'microphone_id', 'speaker_id'],
        num_rows: 108502
    })
    validation: Dataset({
        features: ['meeting_id', 'audio_id', 'text', 'audio', 'begin_time', 'end_time', 'microphone_id', 'speaker_id'],
        num_rows: 13098
    })
    test: Dataset({
        features: ['meeting_id', 'audio_id', 'text', 'audio', 'begin_time', 'end_time', 'microphone_id', 'speaker_id'],
        num_rows: 12643
    })
})

In [ ]:
COLUMNS_TO_KEEP = ['text', 'audio']
all_columns = ami_orig["train"].column_names
columns_to_remove = set(all_columns) - set(COLUMNS_TO_KEEP)
ami = ami_orig.remove_columns(columns_to_remove)

In [ ]:
ami

DatasetDict({
    train: Dataset({
        features: ['text', 'audio'],
        num_rows: 108502
    })
    validation: Dataset({
        features: ['text', 'audio'],
        num_rows: 13098
    })
    test: Dataset({
        features: ['text', 'audio'],
        num_rows: 12643
    })
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(ami["train"].remove_columns(["audio"]), num_examples=10)

,text
0,YEAH
1,BUT WHAT WE WHAT WE NEED IS SOMETHING THAT LOOKS LIKE IT'S FROM THE FUTURE THAT LOOKS COOL THAT'S THAT'S DIFFERENT
2,THAT WE CAN DISCUSS AFTERWARDS
3,I'D SAY THAT TOO
4,BUT IF WE'RE GONNA GO ALONG WITH IT THEN IT DOESN'T NECESSARILY HAVE TO BE LIKE AN APPLE OR SOMETHING LIKE THAT OR A KIWI FRUIT
5,CHIP
6,AH
7,THESE ARE THE FINDINGS WHICH WE GOT AND YEAH ADDING OF A SPEECH INTERFACE IS ALWAYS GOOD FOR A T. V. REMOTE BUT THE TECHNOLOGY
8,I FIGURE WE CAN JUST GO DOWN THE LINE AND ALL THREE OF US CAN HAVE A CHAT ABOUT IT
9,TIME IS RUNNING WE HAVE TO CLOSE THE MEETING IN A FEW MINUTES


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
ami = ami.map(remove_special_characters)

Map:   0%|          | 0/108502 [00:00<?, ? examples/s]

Map:   0%|          | 0/13098 [00:00<?, ? examples/s]

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(ami["train"].remove_columns(["audio"]))

,text
0,but i'm going to look if i've got some examples 'cause perhaps you should choose what's most often used
1,yeah d no i mean do you think there's a risk if it doesn't look like remote control people won't see it as a remote control um and uh
2,fair enough fair en
3,okay
4,and then we can get a docking station
5,are we going to talk now about um the materials that we're gonna use for the case and all those things
6,if you make something that looks like a banana it should have the colour of a banana
7,i'll come to that later
8,um way we're gonna go about doing that is basically at first we're gonna start on the basics
9,number two


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = ami.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ami.column_names["train"])

Map:   0%|          | 0/108502 [00:00<?, ? examples/s]

Map:   0%|          | 0/13098 [00:00<?, ? examples/s]

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'h': 0,
 'v': 1,
 't': 2,
 'n': 3,
 'a': 4,
 'g': 5,
 'e': 6,
 '@': 7,
 ' ': 8,
 'f': 9,
 'k': 10,
 'm': 11,
 'y': 12,
 'o': 13,
 'c': 14,
 'w': 15,
 'l': 16,
 'r': 17,
 'x': 18,
 'z': 19,
 'j': 20,
 'u': 21,
 'd': 22,
 'b': 23,
 "'": 24,
 'q': 25,
 's': 26,
 'i': 27,
 '*': 28,
 'p': 29}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

32

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-base-ami-ihm"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/jainr3/wav2vec2-base-ami-ihm/commit/14730a1590a1a730b5324d8dd38684af3bd4d1e6', commit_message='Upload tokenizer', commit_description='', oid='14730a1590a1a730b5324d8dd38684af3bd4d1e6', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
ami["train"][0]

{'text': 'if you if you s s h and they have this big warning about doing nothing at all in the gateway machine ',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/b093dd855ad7f053bb12157f9ce4b51aaeaf29ccaf78fdc8150b3a8125130e2b/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
  'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
         -0.00244141, -0.00219727], dtype=float32),
  'sampling_rate': 16000}}

In [ ]:
ami["train"][0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b093dd855ad7f053bb12157f9ce4b51aaeaf29ccaf78fdc8150b3a8125130e2b/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
 'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
        -0.00244141, -0.00219727], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(ami["train"]))

print(ami["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(ami["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

okay 


In [ ]:
rand_int = random.randint(0, len(ami["train"]))

print("Target text:", ami["train"][rand_int]["text"])
print("Input array shape:", np.asarray(ami["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", ami["train"][rand_int]["audio"]["sampling_rate"])

Target text: and if neither then we'll have to put something else 
Input array shape: (33280,)
Sampling rate: 16000


In [ ]:
ami_unprepared = ami.copy()

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
ami = ami.map(prepare_dataset, remove_columns=ami.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/108502 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Map (num_proc=4):   0%|          | 0/13098 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Map (num_proc=4):   0%|          | 0/12643 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

In [ ]:
max_input_length_in_sec = 4.0
ami["train"] = ami["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/108502 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-39-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.codevectors', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expe

In [ ]:
model.config.mask_time_prob = 0

In [ ]:
model.freeze_feature_encoder()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

# temporarily select only N examples to see if the whole pipeline works
examples_to_select = list(range(5000))

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ami["train"].select(examples_to_select),
    eval_dataset=ami["test"].select(examples_to_select),
    tokenizer=processor.feature_extractor,
)

In [ ]:
# my test took about 1hr for 1/4 of it
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,4.444000,22.782013,0.999778
1000,2.419400,2.414312,0.927440
1500,1.207100,7.224381,0.869988
2000,0.866000,19.098459,0.747419
2500,0.682600,1.722039,0.698803
3000,0.555500,1.895802,0.715905
3500,0.450300,1.604229,0.645594
4000,0.380300,2.026619,0.680433
4500,0.338600,3.177047,0.643631
5000,0.304500,1.699433,0.617565


/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

KeyboardInterrupt: ignored

In [ ]:
# had to save locally then upload to huggingface...
trainer.save_model(f"/content/gdrive/MyDrive/{repo_name}")

In [ ]:
!git lfs install
!git clone https://huggingface.co/jainr3/wav2vec2-base-ami-ihm

Updated git hooks.
Git LFS initialized.
Cloning into 'wav2vec2-base-ami-ihm'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 8
Unpacking objects: 100% (14/14), 2.80 KiB | 955.00 KiB/s, done.
Filtering content: 100% (2/2), 360.17 MiB | 24.92 MiB/s, done.


In [ ]:
from huggingface_hub import Repository
repo = Repository(local_dir="/content/wav2vec2-base-ami-ihm")

In [ ]:
repo.git_pull()
trainer.push_to_hub()

/content/wav2vec2-base-ami-ihm is already a clone of https://huggingface.co/jainr3/wav2vec2-base-ami-ihm. Make sure you pull the latest changes with `repo.git_pull()`.


Upload file pytorch_model.bin:   0%|          | 32.0k/360M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

To https://huggingface.co/jainr3/wav2vec2-base-ami-ihm
   e61687f..35da288  main -> main

   e61687f..35da288  main -> main

To https://huggingface.co/jainr3/wav2vec2-base-ami-ihm
   35da288..f399697  main -> main

   35da288..f399697  main -> main



'https://huggingface.co/jainr3/wav2vec2-base-ami-ihm/commit/35da2885515df4e3f191a3e78da61858462a728c'

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained(f"jainr3/{repo_name}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(f"jainr3/{repo_name}", 
                                       ctc_loss_reduction="mean", 
                                       pad_token_id=processor.tokenizer.pad_token_id,
                                      ).cuda()

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = ami["test"].map(map_to_result, remove_columns=ami["test"].column_names)

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test WER: {:f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.523622


In [ ]:
show_random_elements(results)

,pred_str,text
0,yeah,yeah
1,alright,alright
2,um like we can just deside sort of thing stat we do wanna do,um like we can just decide sort of things that we do wanna do
3,yeah,yeah
4,yeah because in he start screen you can't have it,yeah because in the start up screen you can't have a
5,yeah,yeah
6,well,well
7,hu,yeah
8,so uah,so yeah
9,so also i don't now h wes send ita and the of nn e bc g an ad afe sence st,so also i don't know if we send it in on the afternoon maybe i can add a few sentences too when i'm done


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(ami["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'[PAD] n n a [PAD] [PAD] | | | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
print(ami_orig["test"][:1]['text'])

['YEAH']


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(ami["validation"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] b u t t | l l i i k k k e | | | [PAD] m y y y | | a a [PAD] b o o [PAD] [PAD] v v e e | | [PAD] f u r r r [PAD] [PAD] n n n [PAD] s s | | h a a v v e | | [PAD] [PAD] [PAD] s s s [PAD] c c r r e e [PAD] [PAD] [PAD] e e e n n n [PAD] s s | | a n d d | t h h a a [PAD] [PAD] t t t | | | [PAD] c c h h h e [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
print(ami_orig["validation"][:1]['text'])

["BUT LIKE MOBILE PHONES HAVE SCREENS AND THEY'RE CHEAP"]


In [ ]:
# way too slow but one way to save files is zip it and download it
!zip -r /content/wav2vec2-base-ami-ihm-checkpoint-5000.zip /content/wav2vec2-base-ami-ihm/checkpoint-5000

  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/ (stored 0%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/rng_state.pth (deflated 28%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/config.json (deflated 67%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/training_args.bin (deflated 48%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/scaler.pt (deflated 55%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/scheduler.pt (deflated 49%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/trainer_state.json (deflated 76%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/optimizer.pt (deflated 8%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/pytorch_model.bin (deflated 9%)
  adding: content/wav2vec2-base-ami-ihm/checkpoint-5000/preprocessor_config.json (deflated 33%)


In [ ]:
from google.colab import files
files.download("/content/wav2vec2-base-ami-ihm-checkpoint-5000.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Note: just loading base model w/o training will not work

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base").cuda()

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.weight', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = ami["test"].map(map_to_result, remove_columns=ami["test"].column_names)

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 6.594


In [ ]:
show_random_elements(results)

,pred_str,text
0,L</s>DZX</s> DZWDL,CJXM'EH</s>E
1,XWUWNUMYNLWNDWDNDWLWMNWMNWCKL PUMND WKSKU NUCWCUWUKDNWNDCU UN DKWU WUYNVMYWMY W</s>S'WKNMDWDYDLWDLWLVMNWMWMCWNWNWCWNCM NMNMDM Y DWMCWCWCWUXUWCGMWYMYVWLWDWDLWDWYMYCWNDWMCWNUDWCWDN</s>W</s>XNCNWXWPWXMNCLNCNW XW</s>NWCW</s>W</s>'XWCDCYWLWUXMGNWNWCWX'W'CW</s>'CGLU</s>NW</s>UNWNDMJNWNWDWNWL WUM,F</s>CECJNHVGNEB</s>WWBNUECEFNENO</s>XSEBTRNUEG</s>MUEKTCJEXEKTCJEXESTCCSNES<s>EMETCEAGECJNEAGECJNEDS</s>LNECJNECJNECGTXM<s>SNEXMUEBC</s>AAE
2,U </s>K</s> </s>S </s> S,LLJLLE
3,NC WCW</s>ND DCDW W</s>U<unk>CPUCWD</s>ULUWNVXWCVCWCXV<unk>CWGUQCN NWCVUDSQMCMG NW<unk>CWPXLNLW WNUQ N Q QWD</s>JQNCD<unk>WDW W DNU LW</s> XDWC</s>UWUW,HNXJETAEKNVGNE<s>MMXEUECJNE ELESECJNMEATMNECJXCVBECJXCEWGFXFSHELX'NBETCENXBTNGE
4,LNW WXUW</s></s>XDUMLW</s>WMLMLWSWXW<unk>WLDNLDLDLWLMLW</s>UW</s>WUW</s>DL LWLWLWLULWNWU WDWDLDWDWDWDW VM W</s>X LWL</s>DNW</s>W</s>WLWVPX,CX'TM<s>ETMCE</s>LEYMBTUNGXCTMECN C</s>GNEXMUEYLAGCETMECJNEJXMUE
5,CW</s>WCU</s>LU</s>JDCN</s>WDL</s>PU NXW </s>CD</s>X NWD</s>OUWCLU CU LNCN</s>GNW WN N QNCWC</s>SCN</s>UQNWND</s>K UNWCUX NCNWC</s>U'LCNWN</s>CW</s>U'LNWCQ QNWCDCW</s>U'LCWC</s>XUWNWCV'XGLNQNC W,XMUETECJTM'ECJNGNVSSEJXDNECEFNEXMCJNGEWGPNYCECNXLEGEXEMNKEWGPNYCEAGE</s>BECES'EXCEXCECJBNE'TMUEAECJTM<s>BE
6,WLNWLUNWLNUWQ DWLSL WC</s>WLW</s>XDWXD</s>NLNWWL</s> NLN W </s>NLNLNWLWN,TEKXBEP</s>BCECJTM'TM<s>EAECXFE
7,CUXCNWCN NW</s>'UGNGNGNGNGNLQLNLGNLN,N XYCSHE
8,</s> </s>,BNNLBE
9,NW </s>N SWCN </s>CNSX</s>NC'<s>'NCKCNCNCNLWULNV<s>NCKCKCKSXNKCNDKQNCVDNCVKNDL</s>CUXQNCNCNCKWN'VCNCW</s>C</s> UG NCXWYNCNCNCNCN MNCWC<s>CDMNMCNQMUQNCMNMNUWCNCNUQLQUQDQU,BETAEH</s>VGNES'TM<s>EAGEKTGNSNBBEKTGNUEKTGNEFSXJEFSXJEFSXJEH</s>EUEKTGNEKTCJEXEBCXGEXMUE</s>LEME


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(ami["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

"| | | | | S X X W X </s> </s> ' C C C C C C C C C C C C C C U N U | | | | | | | U U C C N N H N N N N N N C C C C C C C C C C N C ' H C ' ' N"

### Try facebook/wav2vec2-large-960h - needs to be finetuned to actually work...

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h").cuda()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = ami["test"].map(map_to_result, remove_columns=ami["test"].column_names)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 1.000


In [ ]:
show_random_elements(results)

,pred_str,text
0,,no i think i we don't wanna be tacky if we've got a kind of different shape anyway um the fact the m the material that we would be using would be cheap
1,,and um also with their something like sunny day
2,,the average oh for you
3,,that's right
4,,so i'll try and remember to do that to my own s
5,,you can have it
6,,i would say four
7,,or and you can choose you can go go for go or go for the other
8,,see um yeah
9,,so half of the price would be


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(ami["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'<pad> <pad> <pad> <pad> <pad> N <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> | Y <pad> O U <pad> <pad> <pad> <pad> | | S S <pad> <pad> T <pad> <pad> O <pad> <pad> <pad> P <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> H <pad> E <pad> <pad> R E <pad> | | | | <pad> <pad> <pad> <pad>'

### Trying to record audio and run model

In [ ]:
!pip install ffmpeg-python

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
# TODO load model and run inference...

### Testing inference on a long audio clip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install transformers
!pip -q install youtube_dl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.7 MB/s eta 0:00:00


In [3]:
# may have to restart runtime to get it working
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=5501ec74ee0f2c761a0429b4979e23b9a913adf8e342ce9ba36b0656fb6f68fd
  Stored in directory: /root/.cache/pip/wheels/b7/44/b7/2e7cc9c5fe4a893b9cc83a010d4410557bedf6cf3b5829f497
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.25.1
    Uninstalling imageio-2.25.1:
      Successfully uninstalled imageio-2.25.1


In [4]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from IPython.display import Audio

import moviepy.editor as mp
import torch
import librosa
import os

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2686976/45929032 bytes (5.9%)5144576/45929032 bytes (11.2%)7782400/45929032 bytes (16.9%)10567680/45929032 bytes (23.0%)13467648/45929032 bytes (29.3%)16539648/45929032 bytes (36.0%)19046400/45929032 bytes (41.5%)21725184/45929032 bytes (47.3%)23404544/45929032 bytes (51.0%)25739264/45929032 bytes (56.0%)28647424/45929032 bytes (62.4%)31227904/45929032 bytes (68.0%)

In [13]:
# Load file
filename = "/content/drive/MyDrive/gitlab_engineering_meeting.mp3"

start = 0
end = 600 #24 * 60 + 10

In [6]:
# Load Wav2Vec from huggingface
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
  warnings.warn(



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
clip = mp.AudioFileClip(filename)
end = min(clip.duration, end)

SUBCLIP_SIZE = 10 # sec

clip_paths = []

# Extract Audio-only from mp4
for i in range(start, int(end), SUBCLIP_SIZE):
  print("Processing audio_" + str(i) + ".mp3")
  sub_end = min(i+10, end)
  sub_clip = clip.subclip(i,sub_end)

  sub_clip.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

Processing audio_0.mp3
[MoviePy] Writing audio in audio_0.mp3


100%|██████████| 221/221 [00:00<00:00, 521.00it/s]

[MoviePy] Done.
Processing audio_10.mp3
[MoviePy] Writing audio in audio_10.mp3



100%|██████████| 221/221 [00:00<00:00, 433.45it/s]

[MoviePy] Done.
Processing audio_20.mp3
[MoviePy] Writing audio in audio_20.mp3


...

[MoviePy] Done.
Processing audio_580.mp3
[MoviePy] Writing audio in audio_580.mp3



100%|██████████| 221/221 [00:00<00:00, 503.25it/s]

[MoviePy] Done.
Processing audio_590.mp3
[MoviePy] Writing audio in audio_590.mp3



100%|██████████| 221/221 [00:00<00:00, 485.86it/s]

[MoviePy] Done.


In [15]:
Audio(clip_paths[-1]) # check out the last clip...

In [16]:
cc = ""

for path in clip_paths:
    # Load the audio with the librosa library
    input_audio, _ = librosa.load(path, 
                                sr=16000)

    # Tokenize the audio
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    # Feed it through Wav2Vec & choose the most probable tokens
    with torch.no_grad():
      logits = model(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)

    # Decode & add to our caption string
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    cc += transcription + " "

In [17]:
print(cc)

AY THIS IS ERIC JOHNSON IT'S FEBRUARY EIGHTEEN TWENTY TWENTY ONE AND THIS IS THE ENGINEERING KEY REVIEW AT GITLAB A SO I'VE GOT NUMBER FOUR IN THE AGENDA A WHICH IS A PROPOSAL TO BREAK UP THIS MEETING INTO FOR DEPARTMENT KEYERVI OUS OCCURRENTLY THIS IS ENGINEERING AM DEVELOPMENT QUALITY SECURITY AND U ACTS INFOR STRUCTURE AND AM SUPPORT DO THEIR OWN VIEWS ALREADY I HAVE THE REASONS WHY INCREASE VISIBILITY ABLE TO GO DEEPER A A INCREASE THE OBJECTIVITY WITH WHICH MY REPORTS CAN MANAGE THEIR GROUPS ALLOW ME MORE TIME TO FOCUS ON NEW MARKETS AND A ALLOW ME TO SHIFT IN A MORE OF A QUESTION ASKER MODE THAN GENERATING CONTENT AND ANSWERING QUESTIONS IN THESE MEETINGS A END DA BUT TO AVOID ADDING THREE NETNEW MEETINGS TO STAY COLER'S COUNTERS AT PROPOSE BE DU A SORT OF TWO MONTH ROTATION SO MONTHUON DEVELOPMENT QUALITY GO MONT TO SECURE TE ANUAX WOULD GO HOW DO PEOPLE FEEL ABOUT THAT PROPOSAL THINKIN DE GU COMFER SATIONS IT'S WORKING BEALY WELL UM SO I'M I'M SUPPORTIVE AND THE IS IS THE SMALL

In [ ]:
# timing for transcription (CPU for inference is fine!):
# 2min audio -> 7sec to split + 50sec to do inference 
# 5min audio -> 30sec to split + 125sec to do inference
# 10min audio -> 36sec to split + 215sec to do inference

## References
* https://huggingface.co/datasets/edinburghcstr/ami
* https://huggingface.co/facebook/wav2vec2-large-robust-ft-swbd-300h
* https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb#scrollTo=72737oog2F6U
* https://github.com/huggingface/blog/blob/main/audio-datasets.md
* https://huggingface.co/blog/fine-tune-wav2vec2-english
* https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a#scrollTo=-YilAhGr6gtZ
* https://discuss.huggingface.co/t/valueerror-mask-length-has-to-be-smaller-than-sequence-length-while-finetuning-wav2vec2-0/8295
* https://colab.research.google.com/github/Muennighoff/ytclipcc/blob/main/wav2vec_youtube_captions.ipynb#scrollTo=NXZJng_hvV3H